In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

# Load your data here
data_path = '/kaggle/input/pig-disease-f/data 2/Train'
metadata = pd.read_csv('//kaggle/input/pig-data/data.csv')

class_names = {
    1: 'Pneumonia',
    2: 'PRRS',
    3: 'Swine Fever'
    # Add more class IDs and their corresponding names as needed
}

metadata['class_name'] = metadata['classID'].map(class_names)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Assuming that your audio files are organized in folders named after their classes
class_folders = ['Pnemonia', 'prrs', 'swine_fever']
audio_data = []
labels = []

for folder in class_folders:
    folder_path = os.path.join(data_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            audio, sr = librosa.load(audio_path, sr=None)
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
            audio_data.append(mfccs)
            labels.append(folder)

In [4]:
# Padding MFCCs to ensure uniformity in feature dimensions
max_length = max([mfcc.shape[1] for mfcc in audio_data])
padded_mfccs = [np.pad(mfcc, pad_width=((0, 0), (0, max_length - mfcc.shape[1])), mode='constant') for mfcc in audio_data]

# Converting to NumPy arrays
X = np.array(padded_mfccs)
y = np.array(labels)

# Reshape for LSTM input
X = X.reshape(X.shape[0], X.shape[2], X.shape[1]) # Reshaping to (samples, timesteps, features)

# Label encoding
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 390, 128)          86528     
                                                                 
 dropout (Dropout)           (None, 390, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 136131 (531.76 KB)
Trainable params: 136131 (531.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
num_epochs = 10
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification_lstm.hdf5', verbose=1, save_best_only=True)

start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("Training completed in time: ", datetime.now() - start)

Epoch 1/10
24/26 [==========================>...] - ETA: 0s - loss: 0.8050 - accuracy: 0.7083
Epoch 1: val_loss improved from inf to 0.52609, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 11s 69ms/step - loss: 0.7849 - accuracy: 0.7197 - val_loss: 0.5261 - val_accuracy: 0.8155
Epoch 2/10
 4/26 [===>..........................] - ETA: 0s - loss: 0.5811 - accuracy: 0.8047

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/26 [===========================>..] - ETA: 0s - loss: 0.5547 - accuracy: 0.8087
Epoch 2: val_loss improved from 0.52609 to 0.44545, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.5554 - accuracy: 0.8083 - val_loss: 0.4455 - val_accuracy: 0.8301
Epoch 3/10
25/26 [===========================>..] - ETA: 0s - loss: 0.4416 - accuracy: 0.8462
Epoch 3: val_loss improved from 0.44545 to 0.41763, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.4459 - accuracy: 0.8459 - val_loss: 0.4176 - val_accuracy: 0.8641
Epoch 4/10
24/26 [==========================>...] - ETA: 0s - loss: 0.3497 - accuracy: 0.8854
Epoch 4: val_loss improved from 0.41763 to 0.37296, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.3493 - accuracy: 0.8859 - val_loss: 0.3730 - val_accuracy: 0.8495
Epoch 5/10
25/26 [==================

In [7]:
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy[1]}')

Test accuracy: 0.8155339956283569
